## Extracting amplitude and phase values from CSI complex numbers

### Using cmath library - only useful collumns

In [ ]:
import pandas as pd
import numpy as np
import cmath
from pathlib import Path

# Configuração dos arquivos usando Path
# Defina apenas o nome do arquivo de entrada (sem o caminho completo)
input_filename = "csi_data.csv"
output_filename = "csi_data_processed.csv"

# Obtém o diretório atual do script
current_dir = Path.cwd()

# Cria os caminhos completos para os arquivos
input_file = current_dir / input_filename
output_file = current_dir / output_filename

print(f"Arquivo de entrada: {input_file}")
print(f"Arquivo de saída: {output_file}")
print(f"Diretório atual: {current_dir}")

# Verifica se o arquivo de entrada existe
if not input_file.exists():
    print(f"ERRO: Arquivo {input_filename} não encontrado no diretório atual!")
    print("Arquivos disponíveis no diretório:")
    for file in current_dir.glob("*.csv"):
        print(f"  - {file.name}")
    exit(1)
else:
    print(f"✓ Arquivo de entrada encontrado: {input_filename}")

#Remove useless columns
cols_to_remove = [
    "taget_seq", "taget", "mac", "rate", "mcs", "cwb",
    "smoothing", "not_sounding", "aggregation", "stbc", "sgi",
    "ampdu_cnt", "ant", "rx_state", "agc_gain", "fft_gain"
]

df = pd.read_csv(input_file)

#Processing CSI data
def process_csi(row):
    numbers = list(map(int, row.strip("[]").split(',')))
    numbers = numbers[2:]  #To ignore the first 2 values

    amps, phases = [], []
    for i in range(0, len(numbers), 2):
        real = numbers[i]
        imag = numbers[i + 1]
        c = complex(real, imag)
        amp, phase = cmath.polar(c)
        amps.append(amp)
        phases.append(phase)

    return amps + phases

#Processing data column
csi_processed = df['data'].apply(process_csi)

#Labeling columns
n_subcarriers = len(csi_processed.iloc[0]) // 2
amp_cols = [f"amp_{i}" for i in range(1, n_subcarriers + 1)]
phase_cols = [f"phase_{i}" for i in range(1, n_subcarriers + 1)]
new_columns = amp_cols + phase_cols

#Making DataFrame with amplitude and phase
csi_df = pd.DataFrame(csi_processed.tolist(), columns=new_columns)

#Concatenate with original DataFrame (without data and useless columns)
df_final = pd.concat([df.drop(columns=['data'] + cols_to_remove), csi_df], axis=1)

#Saving output file
df_final.to_csv(str(output_file), index=False)

print(f"Processamento concluído! Arquivo salvo em: {output_file}")
